1. Precision Evaluation
The generate_precision function is designed to assess whether the triples (subject, relation, object) inferred from a knowledge graph are accurately represented in the provided text. This process ensures that the relationships between entities in the knowledge graph are  present within the context of the CVE.

2. Two-Step Recall Evaluation
Given the randomness and limited long-text processing abilities of LLMs, the notebook implements a two-step recall evaluation process to assess whether all relevant knowledge from the text has been extracted:

Step 1 (generate_prompt_recall_1st): Initial Recall Evaluation The first recall function provides an initial assessment of how much relevant information from the text has been successfully captured in the form of triples. It checks whether the ground truth relationships in the text are present in the generated triples, even if the wording or relationships differ slightly.

Step 2 (generate_prompt_2ndtime_checkFalse): Handling False Negatives To address potential false negatives (i.e., missing triples), a second function rechecks the triples flagged as absent in the first step. This second pass verifies whether the relationships between entities are present in the complex knowledge graph, including multi-step links between entities that might have been missed initially.

In [ ]:
def generate_precision(context, result_tocheck, cve_name):
    """
    Generates a prompt to analyze whether the inference results are actually present in the text paragraph.

    Parameters:
    context (str): The text paragraph to be checked.
    result_tocheck (str): The inference result.
    cve_name (str): The name of the CVE being analyzed.

    Returns:
    list: A list containing the generated prompt for analyzing the accuracy of the inference result.
    """
    prompt_message = [
        {
            "role": "user", 
            "content": """I will provide you with an inference result and a text paragraph. Your task is to analyze whether each inference result is actually present in the text paragraph. If the inference result is a triple (i.e., [Entity1, Relation, Entity2]), you need to evaluate each triple individually. If the inference result consists of several sentences, you should evaluate each sentence individually.

            If the inference result uses words like 'attacker', 'vulnerability', or 'victim', you should assume that they refer to the actual CVE name \"""" + cve_name + """\", and evaluate it just like it describes the CVE has the same behavior. You should be a wise judge and consider the inference result is still accurate because it use attacker and other words to replace the CVE name.

            You may find that the inference result states that the CVE name \"""" + cve_name + """\" has some relationship with other entities, but the context only has describe that other entities, and don't other entities with the \"""" + cve_name + """\". That's because the context is only part of the whole content. This entire content is always discussing the same CVE name \"""" + cve_name + """\". You should be a wise judge and consider the context itself is missing the CVE name, and the inference result is still accurate.

            Your output should be in JSON format and include the following keys:
            'Result X': where X is a number starting from 1, incrementing by 1 for each result.
            'Thought For Result X': Your thought process in determining whether the inference result is actually present in the text paragraph. Focus on conveying the same meaning rather than rigidly comparing individual words. As long as the message is clear, exact word-for-word correspondence isn’t necessary. Also, if the inference result is a triple, you should consider the entities and relation separately. For example, if the inference result is [A, B, C], you should evaluate A, B, and C individually.
            'Precision Result For Result X': Your judgment on the inference result, indicating whether it is True or False.

            The keys in the JSON must strictly follow the given format, and you can only change the number X to match the result index. Here is an example:
            ```json
            {"Result 1": "[X, Y, Z]", "Thought For Result 1": "I believe this triple is accurate because...", "Precision Result For Result 1": "True", 
            "Result 2": "A is related to B", "Thought For Result 2": "I believe this statement is accurate because...", "Precision Result For Result 2": "True"}
            ```

            The inference result I’m giving you is: """ + result_tocheck + """, and the text paragraph is: """ + context
        }
    ]
    
    return prompt_message

def generate_prompt_recall_1st(CVE_Name, Generated_Description, Ground_Truth):
    # Define the base content string
    content_template = '''
    I will give you three inputs. Input 1 is the name of a CVE. Input 2 is a description generated by a certain tool. Input 3 is a ground truth description (a list).
    Input 3 consists of natural language sentences that describe the relationship between a CVE and another entity.
    Input 2 is a description generated by a certain tool, presented in a list format, where each element is a triple representing the relationship between two entities, i.e., [A, B, C], where A and C are entities, and B is the relationship word.

    For each sentence in Input 3, you need to determine whether it can be found in Input 2. If found, you should mark it as True in the JSON response, otherwise mark it as False. Even if the relationship word in Input 3 differs from that in Input 2, as long as the entities involved are similar, it should be considered found. Specifically, in Input 2, [A,B,C] represents a relationship B between A and C. If Input 3 describes the relationship between A and C, even if C's relationship word is D, it should still be considered found in Input 2, even if B and D are different. For example, if Input 2 is "[['CVE-2024-0239', 'lead to', 'Reflected Cross-Site Scripting']]" and Input 3 is "CVE-2024-0239 is a Reflected Cross-Site Scripting vulnerability," then the description in Input 3 is considered found in Input 2 because they both describe the relationship between CVE-2024-0239 and Reflected Cross-Site Scripting, even if the relationship words are different and 'Reflected Cross-Site Scripting' and 'Reflected Cross-Site Scripting vulnerability' are not exactly the same, their core entity (Reflected Cross-Site Scripting) is the same.

    Focus on conveying the same meaning rather than rigidly comparing individual words. As long as the message is clear, exact word-for-word correspondence isn’t necessary. Even if the entity names in Input 3 and Input 2 are not identical, as long as they are similar, the description in Input 3 should be considered found in Input 2. For example, if Input 3 mentions "Wordpress 2.0" and Input 2 only mentions "Wordpress," even without the version number, the description in Input 3 should be considered found in Input 2. Similarly, if Input 3 mentions "Wordpress CMS" and Input 2 mentions "CMS," "Wordpress," or "Wordpress vulnerability," the description in Input 3 should also be considered found in Input 2.

    In this task, the descriptions in Input 3 may be found in Input 2 through multi-step links. For example, if the first sentence in Input 3 is "A is related to B," and the description in Input 2 is "[A, use, attacker]," "[attacker, related to, B]," then the sentence in Input 3 can be considered found in Input 2. If the description in Input 2 is "[A, use, D]," "[C (a similar word to D), related to, B]," then the description in Input 3 should also be considered found in Input 2. An example is when Input 2 contains "['CVE-2024-0665', 'be', 'vulnerability'], ['vulnerability', 'be in', 'plugin'], ['plugin', 'be vulnerable to', 'Reflected Cross-Site Scripting']," the corresponding Input 3 sentence "CVE-2024-0239 is a Reflected Cross-Site Scripting vulnerability" is considered found in Input 2 because the description in Input 2 can be linked through multiple steps. Another example is when Input 2 contains "['CVE-2024-24577', 'have', 'CWE-119'], ['CWE-119', 'describe', 'improper restriction of operations within bounds of memory buffer']," the corresponding Input 3 sentence "CVE-2024-24577 is related to memory buffer" is also considered found in Input 2. Another example is when Input 2 contains "['CVE-2024-0239', 'lead to', 'Reflected Cross-Site Scripting'], ['Reflected Cross-Site Scripting', 'target', 'administrator']," the corresponding Input 3 sentence "CVE-2024-0239 targets administrators" is also considered found in Input 2 because the description in Input 2 can be linked through multiple steps.

    In the input, Input 2 may use terms like "attacker" or similar, while Input 3 may use "CVE-XXXX-XXXX." In this case, the description in Input 3 is also considered found in Input 2.
    Focus on conveying the same meaning rather than rigidly comparing individual words. As long as the message is clear, exact word-for-word correspondence isn’t necessary. Try to provide reasons to justify that Ground Truth X was found in the Generated Description.
    
    Your output should be a JSON formatted response, including:
    - "CVE Name": The name of the CVE, which is Input 1
    - "Ground Truth 1": The first sentence of the ground truth description, which is the first sentence of Input 3
    - "Ground Truth 1's reasoning process": The reasoning process explaining why the first sentence was found in the generated description. If you can argue both that it was found and that it wasn't, you should return the reasoning for finding it.
    - "Ground Truth 1 Findable in Generated Description": Whether the first sentence of the ground truth description was found in the generated description. Return True if found, otherwise False.
    - "Ground Truth 2": The second sentence of the ground truth description, which is the second sentence of Input 3
    - "Ground Truth 2 Findable in Generated Description": Whether the second sentence of the ground truth description was found in the generated description. Return True if found, otherwise False.
    ...(and so on)

    Input 1: {CVE_Name}
    Input 2: {Generated_Description}
    Input 3: {Ground_Truth}
    '''
    
    # Format the content
    content = content_template.format(
        CVE_Name=CVE_Name,
        Generated_Description=Generated_Description,
        Ground_Truth=Ground_Truth
    )
    
    # Create the message
    prompt_message = [
        {
            "role": "user",
            "content": content
        }
    ]
    
    return prompt_message

def generate_prompt_2ndtime_checkFalse(CVEGraphData, CVEGroundTruth, CVENAME):
    """
    Generates a prompt message for summarizing another entity and verifying its relationship with the given CVE name.

    Parameters:
    CVEGraphData: Complex network data composed of multiple triples.
    CVEGroundTruth: The ground truth data for the CVE.
    CVENAME: The name of the CVE being processed.

    Returns:
    prompt_message: A JSON-formatted list containing the user's prompt.
    """
    prompt_message = [
        {
            "role": "user", 
            "content": '''First, I will give you a sentence describing the relationship between ''' + CVENAME + ''' and another entity. 
            You need to summarize what the other entity is. For example, "CVE-2024-21390 involves exploiting token validation weaknesses."
            The other entity is 'token validation weaknesses'. "CVE-2024-24024 is related to crafted fileName."
            The other entity is 'crafted fileName'.
            Next, I will give you a complex network composed of multiple triples, and you need to tell me whether the other entity in this network 
            is related to the specific name/attacker/vulnerability of ''' + CVENAME + '''. This relationship can be direct or involve multiple steps. 
            The relationship may differ from the one described in the sentence.
            When providing the route, use the original format, such as [A, B, C]. For example, if I ask whether "CVE-2024-0374" is related 
            to "forged request" in the network, you could answer with [CVE-2024-0374, allow, unauthenticated attacker to create view, 
            [unauthenticated attacker, create, view via forged request]], and then tell me whether it is related.
            Your output should be in JSON format, with the following keys:
            'CVE Name' (the name of the CVE),
            'Another Entity' (the entity you summarized),
            'Possible Route Based on Network' (the route summarizing the relationship between the other entity and the specific name/attacker/vulnerability 
            of ''' + CVENAME + ''' in the network, using the format [A, B, C], where A is the starting point, C is the endpoint, and B is the relationship),
            and 'Is Related' (whether the other entity is related to the specific name/attacker/vulnerability of ''' + CVENAME + ''' in the network; 
            True for related, False for not related).
            Here's an example of the JSON format:
            ```json 
            { "CVE Name": "''' + CVENAME + '''", "Another Entity": "Reflected Cross-Site Scripting", "Possible Route Based on Network": ["''' + CVENAME + '''", "lead to", "Reflected Cross-Site Scripting"], "Is Related": True }
            ```
            Now, the sentence I give you is: ''' + CVEGroundTruth + '''
            The CVE described in the sentence is: ''' + CVENAME + ''', and based on this sentence, you need to summarize the other entity.
            Now, the network I give you is: ''' + CVEGraphData + '''
            My input is complete. You can start answering now.'''
        }
    ]
    
    return prompt_message



